<a href="https://colab.research.google.com/github/hyun-hyang/ResumEmphasizer/blob/main/jd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy_transformers
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 2.0 MB/s 
     |████████████████████████████████| 4.4 MB 15.4 MB/s 
     |████████████████████████████████| 1.1 MB 63.4 MB/s 
     |████████████████████████████████| 101 kB 8.4 MB/s 
     |████████████████████████████████| 596 kB 17.0 MB/s 
     |████████████████████████████████| 6.6 MB 48.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import spacy
from spacy.tokens import DocBin
import json

In [3]:
spacy.__version__

'3.4.1'

In [4]:
!nvidia-smi

Wed Aug 10 17:09:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    32W / 250W |    407MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
!git clone 'https://github.com/hyun-hyang/ResumEmphasizer.git'

Cloning into 'ResumEmphasizer'...
remote: Enumerating objects: 7499, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 7499 (delta 42), reused 120 (delta 36), pack-reused 7368
Receiving objects: 100% (7499/7499), 38.02 MiB | 24.16 MiB/s, done.
Resolving deltas: 100% (812/812), done.


In [6]:
%cd /content/ResumEmphasizer
!git pull

/content/ResumEmphasizer
Already up to date.


In [7]:
cv_data = json.load(open('/content/ResumEmphasizer/Data_annoctated/mergerd jd.json','r'))

In [8]:
len(cv_data)

159

In [9]:
!python -m spacy init fill-config /content/ResumEmphasizer/Data/base_config.cfg /content/ResumEmphasizer/config.cfg

✔ Auto-filled config with all values
✔ Saved config
/content/ResumEmphasizer/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [10]:
cv_data[0]

{'id': 19,
 'label': [[0, 5, 'company'],
  [337, 357, 'project'],
  [362, 418, 'project'],
  [421, 459, 'job'],
  [475, 481, 'required skills'],
  [483, 488, 'required skills'],
  [489, 500, 'required skills'],
  [505, 513, 'required skills'],
  [642, 690, 'project'],
  [695, 772, 'project'],
  [809, 847, 'benefit'],
  [887, 927, 'benefit'],
  [989, 1057, 'benefit'],
  [1160, 1218, 'benefit'],
  [1464, 1532, 'benefit']],
 'text': "Yapz is a cutting edge, growth-stage startup that allows you to find, connect, and co-host events with clubs in your local area. In addition to their schooling, social, family and work lives, college students partake in extracurricular clubs on campus. To allocate enough time to make those clubs successful is extremely difficult. Yapz makes running a club and finding partner clubs for events easy and time efficient.\n\nWe are looking for a Backend Developer who works with Python, flask, SQLAlchemy and Postgres. The developer should also have a practical under

In [11]:
def get_spacy_doc(file, data):
  nlp=spacy.blank('en')
  db=DocBin()

  for item in data:
    text = item['text']
    doc=nlp.make_doc(text)
    annot=item['label']

    
    ents=[]
    entity_indices=[]

    for start, end, label in annot:
      skip_entity=False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity=True
          break
      if skip_entity==True:
         continue
        
      entity_indices = entity_indices + list(range(start, end))

      try:
        span=doc.char_span(start,end,label=label,alignment_mode='contract')
      except:
        continue
        
      if span is None:
        err_data=str([start,end]) + "     " + str(text) + "\n"
        file.write(err_data)
        
      else:
        ents.append(span)

    try:
        doc.ents=ents
        db.add(doc)
    except:
      pass


  return db

In [12]:
from sklearn.model_selection import train_test_split
train, test=train_test_split(cv_data,test_size=0.3)

In [13]:
len(train), len(test)

(111, 48)

In [14]:
file=open('error.txt','w')

db=get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db=get_spacy_doc(file, test)
db.to_disk('test_data.spacy')

file.close()

In [15]:
len(db.tokens)

48

In [16]:
!ls

config.cfg	 jd.ipynb	      README.md     test_data.spacy
Data		 logo		      resume.ipynb  test_jd
Data_annoctated  Persona	      service	    train_data.spacy
error.txt	 Project_Description  test_cv	    wordpress_site


In [17]:
!python -m spacy train /content/ResumEmphasizer/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-08-10 17:10:58,435] [INFO] Set up nlp object from config
[2022-08-10 17:10:58,445] [INFO] Pipeline: ['transformer', 'ner']
[2022-08-10 17:10:58,449] [INFO] Created vocabulary
[2022-08-10 17:10:58,451] [INFO] Finished initializing nlp object
Downloading: 100% 481/481 [00:00<00:00, 484kB/s]
Downloading: 100% 878k/878k [00:00<00:00, 2.39MB/s]
Downloading: 100% 446k/446k [00:00<00:00, 1.43MB/s]
Downloading: 100% 1.29M/1.29M [00:00<00:00, 3.48MB/s]
Downloading: 100% 478M/478M [00:20<00:00, 24.9MB/s]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a mod

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [19]:
nlp = spacy.load('/content/ResumEmphasizer/output/model-best')

In [23]:
!zip -r /content/ResumEmphasizer/best.zip /content/ResumEmphasizer/output/model-best/


  adding: content/ResumEmphasizer/output/model-best/ (stored 0%)
  adding: content/ResumEmphasizer/output/model-best/transformer/ (stored 0%)
  adding: content/ResumEmphasizer/output/model-best/transformer/model (deflated 14%)
  adding: content/ResumEmphasizer/output/model-best/transformer/cfg (stored 0%)
  adding: content/ResumEmphasizer/output/model-best/ner/ (stored 0%)
  adding: content/ResumEmphasizer/output/model-best/ner/moves (deflated 78%)
  adding: content/ResumEmphasizer/output/model-best/ner/model (deflated 8%)
  adding: content/ResumEmphasizer/output/model-best/ner/cfg (deflated 33%)
  adding: content/ResumEmphasizer/output/model-best/vocab/ (stored 0%)
  adding: content/ResumEmphasizer/output/model-best/vocab/strings.json (deflated 73%)
  adding: content/ResumEmphasizer/output/model-best/vocab/lookups.bin (stored 0%)
  adding: content/ResumEmphasizer/output/model-best/vocab/vectors.cfg (stored 0%)
  adding: content/ResumEmphasizer/output/model-best/vocab/key2row (stored 0

In [24]:
from google.colab import files
files.download("/content/ResumEmphasizer/best.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
!pip install PyMuPDF

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.8 MB 7.3 MB/s 


In [21]:
import sys, fitz

In [22]:
for x in range(1,11):
  fname='/content/ResumEmphasizer/test_jd/'+str(x)+'.pdf'
  doc=fitz.open(fname)
  print('jd num '+str(x)+' test')
  text=" "
  for page in doc:
    text=text+str(page.get_text())
  text=text.strip()
  text=' '.join(text.split())
  
  doc=nlp(text)
  for ent in doc.ents:
    print(ent.text, " ->>>>>> ", ent.label_)
  print('\n')


jd num 1 test
Django Developer  ->>>>>>  job
Self managing  ->>>>>>  required skills
comfortable with remote working Desire to learn Basic project management Required  ->>>>>>  required skills
1+ years  ->>>>>>  required year
Python software development  ->>>>>>  required skills
2+ years  ->>>>>>  required year
XML  ->>>>>>  required skills
REST  ->>>>>>  required skills
Web Frameworks  ->>>>>>  required skills
GIT  ->>>>>>  required skills
MySQL  ->>>>>>  required skills
Flask  ->>>>>>  required skills
Django  ->>>>>>  required skills
HTML  ->>>>>>  required skills
CSS  ->>>>>>  required skills
Javascript  ->>>>>>  required skills
JQuery  ->>>>>>  required skills
Bootstrap  ->>>>>>  required skills
UI Design  ->>>>>>  required skills
B.Tech/B.E. - Computers, Electrical, Electronics/Telecommunication,  ->>>>>>  preferred skills
B.Sc  ->>>>>>  required skills


jd num 2 test
DevOps Engineer  ->>>>>>  job
Kubernetes  ->>>>>>  required skills
3-5 yrs  ->>>>>>  required year
Docker  ->>>>>

In [ ]:
# import numpy as np
# import pandas as pd

In [ ]:
# job = []
# for x in range(1,2000):
#   for ent in doc.ents:
#     if(ent.label_ == 'job'):
#       print(ent.text)
#       job.append()
#   print('\n')


In [ ]:
# cols = ("job")
# rows = []
# for ent in doc.ents:
#   if(ent.label_ == 'job'):
#     row = [ent.text]
#     rows.append(row)
# df = pd.DataFrame(rows, columns=cols)
# print(df)